In [1]:
!pip uninstall scipy
!pip install scipy==1.1.0


Uninstalling scipy-1.1.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scipy-1.1.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scipy/*
Proceed (y/n)? y
  Successfully uninstalled scipy-1.1.0
  Using cached https://files.pythonhosted.org/packages/a8/0b/f163da98d3a01b3e0ef1cab8dd2123c34aee2bafbb1c5bffa354cc8a1730/scipy-1.1.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('FCN_pytorch', force_remount=True)

Mounted at FCN_pytorch


In [3]:
!ls
%cd FCN_pytorch/'My Drive'/FCN_pytorch
!ls

FCN_pytorch  sample_data
/content/FCN_pytorch/My Drive/FCN_pytorch
fcn.py		  road_dataset		  run_model.ipynb  utils.py
models		  road_dataset_061119	  scores
output_short.avi  road_dataset_loader.py  train.ipynb
__pycache__	  road_dataset_utils.py   train.py


In [4]:
!ls road_dataset

label_colors.txt  Labeled_idx  labels  src  train.csv  val.csv


In [0]:
%load fcn.py
%load road_dataset_loader.py
%load road_dataset_utils.py
%load utils.py

In [0]:
from fcn import VGGNet, FCN32s, FCN16s, FCN8s, FCNs
from road_dataset_loader import RoadDataset

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import numpy as np
import time
import sys
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [8]:
n_class    = 2
batch_size = 6
epochs     = 25
lr         = 1e-4
momentum   = 0
w_decay    = 1e-5
step_size  = 50
gamma      = 0.5
configs    = "FCNs-BCEWithLogits_batch{}_epoch{}_RMSprop_scheduler-step{}-gamma{}_lr{}_momentum{}_w_decay{}".format(batch_size, epochs, step_size, gamma, lr, momentum, w_decay)
print("Configs:", configs)

root_dir = 'road_dataset'
train_file = os.path.join(root_dir, "train.csv")
val_file   = os.path.join(root_dir, "val.csv")

Configs: FCNs-BCEWithLogits_batch6_epoch25_RMSprop_scheduler-step50-gamma0.5_lr0.0001_momentum0_w_decay1e-05


In [0]:
# create dir for model
model_dir = "models"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
model_path = os.path.join(model_dir, configs)

use_gpu = torch.cuda.is_available()
num_gpu = list(range(torch.cuda.device_count()))

In [0]:
train_data = RoadDataset(csv_file=train_file, phase='train')
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=1)
val_data = RoadDataset(csv_file=val_file, phase='val', flip_rate=0)
val_loader = DataLoader(val_data, batch_size=1, num_workers=1)

In [0]:
vgg_model = VGGNet(requires_grad=True, remove_fc=True)
fcn_model = FCNs(pretrained_net=vgg_model, n_class=n_class)

In [12]:
if use_gpu:
    ts = time.time()
    vgg_model = vgg_model.cuda()
    fcn_model = fcn_model.cuda()
    fcn_model = nn.DataParallel(fcn_model, device_ids=num_gpu)
    print("Finish cuda loading, time elapsed {}".format(time.time() - ts))

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.RMSprop(fcn_model.parameters(), lr=lr, momentum=momentum, weight_decay=w_decay)
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)  # decay LR by a factor of 0.5 every 30 epochs


Finish cuda loading, time elapsed 3.8312454223632812


In [0]:
# create dir for score
score_dir = os.path.join("scores", configs)
if not os.path.exists(score_dir):
    os.makedirs(score_dir)
IU_scores    = np.zeros((epochs, n_class))
pixel_scores = np.zeros(epochs)

In [0]:
def train():
    for epoch in range(epochs):
        scheduler.step()

        ts = time.time()
        for iter, batch in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = Variable(batch['X'].cuda())
                labels = Variable(batch['Y'].cuda())
            else:
                inputs, labels = Variable(batch['X']), Variable(batch['Y'])

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        #torch.save(fcn_model, model_path)
        out_filename = '091119_db2_epoch' + str(epoch)
        torch.save(fcn_model, out_filename)

        val(epoch)


def val(epoch):
    fcn_model.eval()
    total_ious = []
    pixel_accs = []
    for iter, batch in enumerate(val_loader):
        if use_gpu:
            inputs = Variable(batch['X'].cuda())
        else:
            inputs = Variable(batch['X'])

        output = fcn_model(inputs)
        output = output.data.cpu().numpy()

        N, _, h, w = output.shape
        pred = output.transpose(0, 2, 3, 1).reshape(-1, n_class).argmax(axis=1).reshape(N, h, w)

        target = batch['l'].cpu().numpy().reshape(N, h, w)
        for p, t in zip(pred, target):
            total_ious.append(iou(p, t))
            pixel_accs.append(pixel_acc(p, t))

    # Calculate average IoU
    total_ious = np.array(total_ious).T  # n_class * val_len
    ious = np.nanmean(total_ious, axis=1)
    pixel_accs = np.array(pixel_accs).mean()
    print("epoch{}, pix_acc: {}, meanIoU: {}, IoUs: {}".format(epoch, pixel_accs, np.nanmean(ious), ious))
    IU_scores[epoch] = ious
    np.save(os.path.join(score_dir, "meanIU"), IU_scores)
    pixel_scores[epoch] = pixel_accs
    np.save(os.path.join(score_dir, "meanPixel"), pixel_scores)


# borrow functions and modify it from https://github.com/Kaixhin/FCN-semantic-segmentation/blob/master/main.py
# Calculates class intersections over unions
def iou(pred, target):
    ious = []
    for cls in range(n_class):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = pred_inds[target_inds].sum()
        union = pred_inds.sum() + target_inds.sum() - intersection
        if union == 0:
            ious.append(float('nan'))  # if there is no ground truth, do not include in evaluation
        else:
            ious.append(float(intersection) / max(union, 1))
        # print("cls", cls, pred_inds.sum(), target_inds.sum(), intersection, float(intersection) / max(union, 1))
    return ious


def pixel_acc(pred, target):
    correct = (pred == target).sum()
    total   = (target == target).sum()
    return correct / total

In [19]:
val(0)

epoch0, pix_acc: 0.8957373046875, meanIoU: 0.7926652536144199, IoUs: [0.86387868 0.72145182]


In [20]:
train()

epoch0, iter0, loss: 0.32116055488586426
epoch0, iter10, loss: 0.30609259009361267
epoch0, iter20, loss: 0.12424380332231522
epoch0, iter30, loss: 0.11585499346256256
Finish epoch 0, time elapsed 33.79305696487427


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DataParallel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FCNs. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type VGGNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6

epoch0, pix_acc: 0.9160816107855903, meanIoU: 0.8305118668435989, IoUs: [0.88990878 0.77111495]
epoch1, iter0, loss: 0.17490147054195404
epoch1, iter10, loss: 0.12172290682792664
epoch1, iter20, loss: 0.15919002890586853
epoch1, iter30, loss: 0.1944122314453125
Finish epoch 1, time elapsed 33.83030986785889
epoch1, pix_acc: 0.9235679117838541, meanIoU: 0.8454308714342342, IoUs: [0.89946205 0.79139969]
epoch2, iter0, loss: 0.06878086924552917
epoch2, iter10, loss: 0.13450887799263
epoch2, iter20, loss: 0.1493391990661621
epoch2, iter30, loss: 0.10135672986507416
Finish epoch 2, time elapsed 33.83292102813721
epoch2, pix_acc: 0.9683957248263889, meanIoU: 0.9349108084690982, IoUs: [0.95173174 0.91808987]
epoch3, iter0, loss: 0.12297239154577255
epoch3, iter10, loss: 0.1498040407896042
epoch3, iter20, loss: 0.05057039111852646
epoch3, iter30, loss: 0.07747229933738708
Finish epoch 3, time elapsed 33.81836485862732
epoch3, pix_acc: 0.9672476535373263, meanIoU: 0.931312256849967, IoUs: [0.94